<a href="https://colab.research.google.com/github/AnudeepReddy-Katta/RNNs-LSTM-Attention/blob/main/Part_2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np
import pandas as pd

import random
import math
import time

In [9]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
!pip install spacy --upgrade

Requirement already up-to-date: spacy in /usr/local/lib/python3.7/dist-packages (3.0.6)


In [10]:
%%bash
python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2021-06-24 07:16:46.764204: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [11]:
spacy_en = spacy.load('en_core_web_sm')

In [12]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [13]:
data = pd.read_csv("/content/drive/MyDrive/s7/quora_duplicate_questions.tsv", sep='\t')

In [14]:
print(data.shape)
print(data.columns)

(404290, 6)
Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'], dtype='object')


In [15]:
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [16]:
data = data[data['is_duplicate'] == 1]
print(data.shape)
data.head()

(149263, 6)


,id,qid1,qid2,question1,question2,is_duplicate
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1


In [17]:
data = data.drop(columns=['id', 'qid1', 'qid2','is_duplicate'])

In [18]:
data.question1 = np.array(data.question1).astype(str)
data.question2 = np.array(data.question2).astype(str)

In [19]:
question1 = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

question2 = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [20]:
from sklearn.model_selection import train_test_split

train, test_data = train_test_split(data, test_size=0.3)
train = train.reset_index().drop(columns=['index'])
test_data = test_data.reset_index().drop(columns=['index'])
train_data, valid_data = train_test_split(train, test_size=0.25)
train_data = train_data.reset_index().drop(columns=['index'])
valid_data = valid_data.reset_index().drop(columns=['index'])

In [21]:
print(train_data.shape)
print(valid_data.shape)
print(test_data.shape)

(78363, 2)
(26121, 2)
(44779, 2)


In [22]:
fields = [('question1', question1),('question2',question2)]

In [23]:
example = [torchtext.legacy.data.Example.fromlist([train_data.question1[i],train_data.question2[i]], fields) for i in range(train_data.shape[0])]

In [24]:
trainDataset = torchtext.legacy.data.Dataset(example, fields)

In [25]:
example = [torchtext.legacy.data.Example.fromlist([test_data.question1[i],test_data.question2[i]], fields) for i in range(test_data.shape[0])]

In [26]:
testDataset = torchtext.legacy.data.Dataset(example, fields)

In [27]:
example = [torchtext.legacy.data.Example.fromlist([valid_data.question1[i],valid_data.question2[i]], fields) for i in range(valid_data.shape[0])]

In [28]:
validDataset = torchtext.legacy.data.Dataset(example, fields)

In [29]:
print(f"Number of training examples: {len(trainDataset.examples)}")
print(f"Number of validation examples: {len(validDataset.examples)}")
print(f"Number of testing examples: {len(testDataset.examples)}")

Number of training examples: 78363
Number of validation examples: 26121
Number of testing examples: 44779


In [30]:
print(vars(trainDataset.examples[0]))

{'question1': ['what', 'is', 'the', 'process', 'of', 'financing', 'in', 'acquisition', 'process', '?'], 'question2': ['what', 'is', 'the', 'process', 'of', 'financing', 'in', 'acquisition', '?']}


In [31]:
question1.build_vocab(trainDataset)
question2.build_vocab(trainDataset)

In [32]:
print(f"Unique tokens in Question vocabulary: {len(question1.vocab)}")
print(f"Unique tokens in Answer vocabulary: {len(question2.vocab)}")

Unique tokens in Question vocabulary: 21290
Unique tokens in Answer vocabulary: 21201


In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [34]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (trainDataset, validDataset, testDataset), 
    batch_size = BATCH_SIZE, sort=False, 
    device = device)

In [35]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [36]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [37]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [38]:
INPUT_DIM = len(question1.vocab)
OUTPUT_DIM = len(question2.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [39]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(21290, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(21201, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=21201, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [40]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 29,110,225 trainable parameters


In [41]:
optimizer = optim.Adam(model.parameters())

In [42]:
question2_PAD_IDX = question2.vocab.stoi[question2.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = question2_PAD_IDX)

In [43]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.question1
        trg = batch.question2
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [44]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.question1
            trg = batch.question2

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [45]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [46]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 5m 10s
	Train Loss: 5.226 | Train PPL: 185.967
	 Val. Loss: 5.261 |  Val. PPL: 192.747
Epoch: 02 | Time: 5m 10s
	Train Loss: 4.470 | Train PPL:  87.380
	 Val. Loss: 4.965 |  Val. PPL: 143.331
Epoch: 03 | Time: 5m 12s
	Train Loss: 3.915 | Train PPL:  50.130
	 Val. Loss: 4.553 |  Val. PPL:  94.877
Epoch: 04 | Time: 5m 11s
	Train Loss: 3.522 | Train PPL:  33.867
	 Val. Loss: 4.311 |  Val. PPL:  74.488
Epoch: 05 | Time: 5m 11s
	Train Loss: 3.237 | Train PPL:  25.446
	 Val. Loss: 4.219 |  Val. PPL:  67.968
Epoch: 06 | Time: 5m 12s
	Train Loss: 3.000 | Train PPL:  20.089
	 Val. Loss: 4.139 |  Val. PPL:  62.762
Epoch: 07 | Time: 5m 13s
	Train Loss: 2.817 | Train PPL:  16.733
	 Val. Loss: 4.089 |  Val. PPL:  59.691
Epoch: 08 | Time: 5m 11s
	Train Loss: 2.660 | Train PPL:  14.295
	 Val. Loss: 4.060 |  Val. PPL:  58.001
Epoch: 09 | Time: 5m 11s
	Train Loss: 2.538 | Train PPL:  12.653
	 Val. Loss: 4.018 |  Val. PPL:  55.595
Epoch: 10 | Time: 5m 9s
	Train Loss: 2.407 | Train PPL:

In [47]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.001 | Test PPL:  54.665 |
